In [1]:
from caveclient import CAVEclient
import numpy as np 
import pandas as pd

import sys
sys.path.append('utils')
from FANC_utils import filter_syn_df, compute_connectivity_matrix

In [2]:
# Access FlyWire public dataset 
client = CAVEclient()
datastack_name = 'flywire_fafb_public'
client = CAVEclient(datastack_name)

In [3]:
# IDs of neurons of interest

# Web neuron
ids = [720575940636656632] # Left brain

In [4]:
# Get neuron information
neurons_df = client.materialize.query_table('proofread_neurons')

In [5]:
# Get neuron classifications 
class_df = client.materialize.query_table('hierarchical_neuron_annotations',
                                          filter_in_dict={'classification_system': ['super_class']})

In [6]:
# Get presynaptic partners 
syn_df = client.materialize.query_view('valid_synapses_nt_np', 
                                       filter_in_dict={'post_pt_root_id': ids})

In [8]:
# Remove connections less than syn_thresh from syn_df
syn_thresh = 5
syn_df = filter_syn_df(syn_df, syn_thresh)

In [9]:
# Remove IDs belonging to fragments (non-proofread neurons) 
proofread_neuron_ids = neurons_df.pt_root_id.unique().tolist()
idx_to_exclude = []
for idx, row in syn_df.iterrows():
    if row['pre_pt_root_id'] not in proofread_neuron_ids:
        idx_to_exclude.append(idx)

syn_df = syn_df.drop(idx_to_exclude)

In [10]:
# Compute connectivity matrix between neuron(s) of interest and partner neurons
conn_mat, pre_ids, post_ids = compute_connectivity_matrix(syn_df)
conn_mat = np.transpose(conn_mat)

In [11]:
# Get IDs of partner neurons 
partner_ids = pre_ids # Same as filt_syn_df.pre_pt_root_id.unique().tolist()

## Synapse counts per cell type

In [12]:
# Get cell types of partner neurons
partner_cell_types = []
for ix, i in enumerate(partner_ids):
    partner_cell_types.append(class_df.cell_type[class_df.pt_root_id==i].item())

In [13]:
# Initialize dict with synapse counts per cell type
n_syn_cell_type = {}
n_neurons_cell_type = {}
for i in list(np.unique(partner_cell_types)):
    n_syn_cell_type[i] = 0
    n_neurons_cell_type[i] = 0
    
# Populate dict based on conn_mat
normalize_max = np.sum(conn_mat)
for ix, i in enumerate(partner_ids):
    n_syn_cell_type[partner_cell_types[ix]]  += np.sum(conn_mat[:,ix])/normalize_max
    n_neurons_cell_type[partner_cell_types[ix]] += 1

In [14]:
n_syn_cell_type

{'ascending': 0.2664706899533116,
 'central': 0.4803735085595712,
 'descending': 0.2531558014871175}

In [15]:
n_neurons_cell_type

{'ascending': 74, 'central': 59, 'descending': 37}

## Synapse counts per neuropil

In [16]:
# Get presynaptic partners of partner neurons
syn_partner_df = client.materialize.query_view('valid_synapses_nt_np', 
                                               filter_in_dict={'post_pt_root_id': partner_ids})

In [18]:
# Remove connections less than syn_thresh from syn_partner_df
syn_thresh = 5
syn_partner_df = filter_syn_df(syn_partner_df, syn_thresh)

In [19]:
syn_partner_df['syn_count'] = 1

In [20]:
# Generate list of neuropil names with _L and _R
neuropil_names_merged = syn_partner_df.neuropil.to_list()
for ix, i in enumerate(neuropil_names_merged):
    if '_' in i:
        neuropil_names_merged[ix] = i[:-(len(i)-i.find('_'))] 

# Replace neuropil names 
syn_partner_df['neuropil'] = pd.Series(neuropil_names_merged).values
syn_partner_df.reset_index(drop=True, inplace=True)

# Sum entries where neuropil is the same. Keep pre_root_id and post_root_id.
syn_partner_df = syn_partner_df.groupby(['neuropil','pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index()

,neuropil,pre_pt_root_id,post_pt_root_id,syn_count
0,AL,720575940612395178,720575940620032379,1
1,AL,720575940623862015,720575940620032379,4
2,AL,720575940627281594,720575940620032379,2
3,AL,720575940627281594,720575940622669971,2
4,AL,720575940629489260,720575940622669971,1
...,...,...,...,...
20870,WED,720575940659063169,720575940624101524,8
20871,WED,720575940659063169,720575940626089502,1
20872,WED,720575940659264385,720575940620860398,9
20873,WED,720575940659264385,720575940623084170,20


In [22]:
# Generate dataframe with relative synapse counts from partner neuropils
conn_mat_neuropil = []

for ix, i in enumerate(ids):
    for jx, j in enumerate(partner_ids):
        n_syn_noi_partner = conn_mat[ix,jx]
        
        if n_syn_noi_partner>0:
            # post_root_ids are partner neurons, pre_root_ids are neurons upstream of partner neurons
            temp = syn_partner_df[syn_partner_df.post_pt_root_id==j]
                
            # Sum syn_count for each neuropil
            temp_summed = temp.groupby(['post_pt_root_id','neuropil'])['syn_count'].sum().reset_index()
            # Add to data structure
            for kx, k in temp_summed.iterrows():
                conn_mat_neuropil.append([i, # noi 
                                          j, # partner
                                          temp_summed.neuropil[kx], # neuropil of partner 
                                          temp_summed.syn_count[kx]/np.sum(temp_summed.syn_count), # relative syn_count of partner in neuropil
                                          n_syn_noi_partner, # syn_count between noi and partner
                                          temp_summed.syn_count[kx]/np.sum(temp_summed.syn_count) * n_syn_noi_partner]) # weight syn_count of partner            

# Generate dataframe 
df_conn_neuropil = pd.DataFrame(conn_mat_neuropil)
df_conn_neuropil.columns = ["noi_id", "partner_id", "neuropil", "rel_syn_count_partner", "syn_count_noi_partner", "weighted_syn_count_partner"]      
df_conn_neuropil

,noi_id,partner_id,neuropil,rel_syn_count_partner,syn_count_noi_partner,weighted_syn_count_partner
0,720575940636656632,720575940639428174,AVLP,0.358876,98.0,35.169860
1,720575940636656632,720575940639428174,GNG,0.054917,98.0,5.381865
2,720575940636656632,720575940639428174,GOR,0.134100,98.0,13.141762
3,720575940636656632,720575940639428174,IB,0.010217,98.0,1.001277
4,720575940636656632,720575940639428174,ICL,0.307791,98.0,30.163474
...,...,...,...,...,...,...
814,720575940636656632,720575940629489414,SPS,0.083333,5.0,0.416667
815,720575940636656632,720575940629489414,VES,0.031008,5.0,0.155039
816,720575940636656632,720575940629489414,WED,0.042636,5.0,0.213178
817,720575940636656632,720575940620473547,GNG,1.000000,5.0,5.000000


In [23]:
# Sum weights for each neuopil
df_conn_neuropil_summed = df_conn_neuropil.groupby(['neuropil'])['weighted_syn_count_partner'].sum().reset_index().sort_values('weighted_syn_count_partner', ascending=False)
df_conn_neuropil_summed['rel'] = df_conn_neuropil_summed.weighted_syn_count_partner/df_conn_neuropil_summed.weighted_syn_count_partner.sum()
df_conn_neuropil_summed

,neuropil,weighted_syn_count_partner,rel
12,GNG,3267.473748,0.607224
4,AVLP,467.330974,0.086848
24,SAD,375.474211,0.069778
15,ICL,359.633095,0.066834
13,GOR,162.846169,0.030263
23,PVLP,151.993239,0.028246
26,SIP,73.861376,0.013726
30,VES,68.926243,0.012809
10,FLA,58.830290,0.010933
25,SCL,54.805156,0.010185
